<a href="https://colab.research.google.com/github/ensarkaya/CS425-Group7-TermProject/blob/master/myVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import csv
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
#from six import string_types
path = '/content/gdrive/My Drive/DataSets/1million.csv'
df = pd.read_csv(path)
size =998261

In [3]:
nltk.download('stopwords')
dummy =["and","or","is","are","what","where","who","whom","when","in","cannot","with"
            ,"a","an","of","to","for","from","by","can","i","with","its","can't","I",
             "there","that","thats","all","does","do","did","the","am","using","it",
             "trying","how","be","not","doesn't","don't","didn't","won't","will",
             "my","be","should","would","as","use","must","have","but","however","at",
             "another","other","one","two","three","four","five","still","on","why","it"
              ,"how","a","to","'",".",",",";",":","?","!","-","_","1","2","3","4","5","6","7","8","9","0"]
stop = stopwords.words('english')
stop.append(dummy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
for i in range(size):
  str1 = df.loc[i,'body']
  soup = BeautifulSoup(str1)
  for a in soup.find_all('p'):
    if a.string is None:
      df.loc[i,'body']="None"  
    else:
      df.loc[i,'body']=a.string

In [0]:
from six import string_types
df["title"] = df["title"].apply(lambda x: x.lower())
df["title"] = df["title"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
df["tags"] = df["tags"].apply(lambda x: x.lower() if(isinstance(x, string_types)) else "-")
df["tags"] = df["tags"].str.split('|').apply(lambda x: ' '.join(k for k in x if k not in dummy))
df["tags"] = df["tags"].str.split('-').apply(lambda x: ' '.join(k for k in x if k not in dummy))
#df["body"] = df["body"].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

In [0]:
#keywords -> id's of questions
dic = {}
#questionId -> (score, answerCount, viewCount, comment_count)
values = {}

for i in range(size):
  values[df.loc[i,'id']] = (df.loc[i,'title'], df.loc[i,'score'], df.loc[i,'answer_count'], df.loc[i,'view_count'], df.loc[i,'comment_count'])
  #print("idler : ",df.loc[i,'id'])
  #print("valueler : ",values[df.loc[i,'id']])
  df.loc[i,'title']#unneccessary
  string1 = df.loc[i,'title'].split()
  string2= df.loc[i,'tags'].split()
  #string3=df.loc[i,'body'].split()
  string=string1+string2#+string3
  for j in range(len(string)):
    while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
    while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
    if  string[j] != "":
      if string[j] in dic:
        dic[string[j]].append(df.loc[i,'id'])
      else:
        dic[string[j]]=[df.loc[i,'id']]


In [0]:
dic2 = {} #keywords -> keyword count
dic3 = {} #questionId -> keywords if greater than threshold
dic4 = {} #keyword -> index
dic5 = {} #questionId -> keywords as indices
keywordCount=0
threshold = 2; 
count2=0
for i in dic:
#   print(i)
  count=0
  key = i
  
  for j in dic[i]:
    count+=1
  dic2[i]=count
#   print(i," **** ", count)
  if count > threshold:
    keywordCount=keywordCount+1
    for j in range(count):
      id = dic[i][j] 
      if id in dic3:
        if i not in dic3[id]:
          dic3[id].append(i)
      else:
        dic3[id] = [i]
    if key not in dic4:
      dic4[key] = count2
      count2+=1

In [8]:
#how many keywords do we have
print(keywordCount)

51803


In [0]:
for i in dic3:
  List = []
  for j in dic3[i]:
    if j in dic4:
      List.append(dic4[j])
  dic5[i] = List
       

In [0]:
import random
funcSize=128
randomNum={}
for i in range(funcSize):
  a=random.randint(1,50000)
  b=random.randint(1,50000)
  randomNum[i]=(a,b)

In [0]:
col=len(dic5)
row=funcSize
matrix = [[0] * col for i in range(row)]
reverseId = {}
for i in range(funcSize):
  count=0
  for j in dic5:
    minSoFar=9999999
    for k in dic5[j]:
      hashval=(randomNum[i][0]*k + randomNum[i][1]) % 51131
      if hashval<minSoFar:
        minSoFar = hashval
    matrix[i][count]=minSoFar
    #reverse[0] -> excel 0. satırdaki Q IDsi
    reverseId[count] = j
    count +=1  

In [0]:
file2 = open("signature.txt", "w")

for i in range(funcSize):
    for j in range(size):
      file2.write("%d" % matrix[i][j])
      file2.write(" ")
    file2.write("\n")

In [12]:
#keywords
question = input()
s = question.split(" ")
string = [x for x in s if x not in dummy]

print(string)

for j in range(len(string)):
  string[j] = string[j].lower()
  while len(string[j]) > 0 and (string[j][0] < 'a' or string[j][0] > 'z') and (string[j][0] < '0' or string[j][0] > '9') and string[j][0] != '#' and string[j][0] != '+':
      string[j] = string[j][1:]
  while len(string[j]) > 0 and (string[j][len(string[j])-1] < 'a' or string[j][len(string[j])-1] > 'z') and (string[j][len(string[j])-1] < '0' or string[j][len(string[j])-1] > '9') and string[j][len(string[j])-1] != '#' and string[j][len(string[j])-1] != '+':
      string[j] = string[j][:len(string[j])-1]
      
      
print("final string : ", string)      

if(len(string) <= 0):
  print('Question not valid.\n')
else:
    #calculating the metascore
        
    #min-hash of the keywords
    indexQ = []
    for i in string:
      if i in dic4:
        indexQ.append(dic4[i]);
    print("index : ", indexQ)
    minHash = []
    for i in randomNum:
      minValue = 50311
      for j in indexQ:
        hashValue = (randomNum[i][0]*j + randomNum[i][1]) % 51131
        if(hashValue < minValue):
          minValue = hashValue
      minHash.append(minValue)
      
    #print("minHash list : ",minHash)
    
    #locality sensitive hashing
    orders=[]
    candidates = []
    r = 4
    band = funcSize // r;
    
    
    print("band : ",band)
    
    index = 0;
    for i in range(band):
      string = ''
      for j in range(r):
        string = string + str(minHash[i*r + j])
        hashvalue = hash(string)
        #print("string check :", string, "\n")
        #print("hashvalue : ", hashvalue)
      for j in range(col):
        string = ''
        for k in range(r):
          string = string + str(matrix[i*r + k][j])
          if(hash(string) == hashvalue):
            if reverseId[j] not in candidates:
              candidates.append( reverseId[j] )
              orders.append(j)
              print("candidate id: ", reverseId[j]," Question: ", dic3[reverseId[j]])

    
    #output
    score = 0;
    answerCount = 0
    viewCount = 0
    commentCount = 0
    for id in candidates:
      #print("candidate id : ",id)
      score += values[id][1]
      answerCount += values[id][2]
      viewCount += values[id][3]
      commentCount += values[id][4]
    length = len(candidates)
    if (length == 0):
        print ("Completely new question!")
    else:
      print ("Predicted score: ", score/length, "\n")
      print ("Predicted number of answers: ", answerCount/length, "\n")
      print ("Predicted view count: ", viewCount/length, "\n")
      print ("Predicted number of comments: ", commentCount/length, "\n")

java string compile error method
['java', 'string', 'compile', 'error', 'method']
final string :  ['java', 'string', 'compile', 'error', 'method']
index :  [134, 23, 1119, 96, 26]
band :  32
candidate id:  51734906  Question:  ['string', 'android', 'error', 'java', 'firebase', 'database', 'hashmap', 'message', 'realtime']
candidate id:  51689843  Question:  ['string', 'error', 'java', 'input', 'collecting']
candidate id:  50346269  Question:  ['string', 'error', 'java', 'passing', 'entering', 'vlaue']
candidate id:  49515271  Question:  ['string', 'method', 'java', 'use', 'caller']
candidate id:  49126461  Question:  ['string', 'method', 'java', 'reverse']
candidate id:  49810801  Question:  ['string', 'method', 'java', 'arrays', 'create', 'password']
candidate id:  51038181  Question:  ['string', 'method', 'java']
candidate id:  49897844  Question:  ['string', 'method', 'java', 'execute']
candidate id:  49756254  Question:  ['c#', 'string', 'java', 'decode', 'base64', 'encode']
candid

In [13]:
lst=[]
#Jaccard Similarity
for x in range(len(candidates)):
  list1=[]
  list1 = dic5[reverseId[orders[x]]] 
  list2= indexQ
  intersection=0
  for y in range(len(list2)):
     if list2[y] in list1:
      intersection=intersection+1
  union = (len(list1) + len(list2)) - intersection
  sim = float(intersection / union)
  lst.append((reverseId[orders[x]],sim))
  #print("list1: ", list1, " list2: ",list2)
  print("list1: ", list1, " list2: ",list2)
  print("Sim= ",sim, " id: ",reverseId[orders[x]], "intersection:", intersection, "union: ", union)

list1:  [23, 83, 96, 134, 170, 197, 922, 1087, 1089]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.2727272727272727  id:  51734906 intersection: 3 union:  11
list1:  [23, 96, 134, 897, 5342]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.42857142857142855  id:  51689843 intersection: 3 union:  7
list1:  [23, 96, 134, 1251, 6084, 44641]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.375  id:  50346269 intersection: 3 union:  8
list1:  [23, 26, 134, 547, 3526]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.42857142857142855  id:  49515271 intersection: 3 union:  7
list1:  [23, 26, 134, 2885]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.5  id:  49126461 intersection: 3 union:  6
list1:  [23, 26, 134, 135, 198, 473]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.375  id:  49810801 intersection: 3 union:  8
list1:  [23, 26, 134]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.6  id:  51038181 intersection: 3 union:  5
list1:  [23, 26, 134, 1365]  list2:  [134, 23, 1119, 96, 26]
Sim=  0.5  id:  49897844 intersection: 3 union:  6
l

In [0]:
sim=float(1/band)**(1/r)
print(sim)
print(band," ",r)

0.42044820762685725
32   4


AttributeError: ignored